<a href="https://colab.research.google.com/github/tensorbytes0202/Machine-learning/blob/main/Column_transformer.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import numpy as np
import pandas as pd

In [ ]:
from sklearn.impute import SimpleImputer
from sklearn.preprocessing import OneHotEncoder
from sklearn.preprocessing import OrdinalEncoder
from sklearn.compose import ColumnTransformer
from sklearn.pipeline import Pipeline

In [ ]:
df = pd.read_csv('/content/covid_toy.csv')

In [ ]:
df.head()

,age,gender,fever,cough,city,has_covid
0,60,Male,103.0,Mild,Kolkata,No
1,27,Male,100.0,Mild,Delhi,Yes
2,42,Male,101.0,Mild,Delhi,No
3,31,Female,98.0,Mild,Kolkata,No
4,65,Female,101.0,Mild,Mumbai,No


In [ ]:
df.sample(8)

,age,gender,fever,cough,city,has_covid
33,26,Female,98.0,Mild,Kolkata,No
31,83,Male,103.0,Mild,Kolkata,No
86,25,Male,104.0,Mild,Bangalore,Yes
27,33,Female,102.0,Strong,Delhi,No
67,65,Male,99.0,Mild,Bangalore,No
3,31,Female,98.0,Mild,Kolkata,No
26,19,Female,100.0,Mild,Kolkata,Yes
28,16,Male,104.0,Mild,Kolkata,No


In [ ]:
from sklearn.model_selection import train_test_split
X_train,X_test,y_train,y_test = train_test_split(df.drop(columns=['has_covid']),df['has_covid'],
                                                test_size=0.2)

In [ ]:
df['cough'].value_counts()

,count
cough,
Mild,62
Strong,38


In [ ]:
df['city'].value_counts()

,count
city,
Kolkata,32
Bangalore,30
Delhi,22
Mumbai,16


In [ ]:
  df.isnull().sum()

,0
age,0
gender,0
fever,10
cough,0
city,0
has_covid,0


In [ ]:
X_train

,age,gender,fever,cough,city
75,5,Male,102.0,Mild,Kolkata
52,47,Female,100.0,Strong,Bangalore
84,69,Female,98.0,Strong,Mumbai
9,64,Female,101.0,Mild,Delhi
50,19,Male,101.0,Mild,Delhi
...,...,...,...,...,...
79,48,Female,103.0,Mild,Kolkata
33,26,Female,98.0,Mild,Kolkata
4,65,Female,101.0,Mild,Mumbai
23,80,Female,98.0,Mild,Delhi


Without Column transformer(Simple imputer)

In [ ]:
#adding Simple imputer to fever col
si = SimpleImputer()
X_train_fever = si.fit_transform(X_train[['fever']])

#also the test data
X_test_fever = si.fit_transform(X_test[['fever']])
X_train_fever.shape
#jo fever wale col me missing value th wo mean se replace ho gyi isme

(80, 1)

Ordinal encoding For cough

In [ ]:
# Ordinalencoding -> cough
oe = OrdinalEncoder(categories=[['Mild','Strong']])
X_train_cough = oe.fit_transform(X_train[['cough']])

# also the test data
X_test_cough = oe.fit_transform(X_test[['cough']])

X_train_cough.shape

(80, 1)

One hot encoding For gender, city

In [ ]:
ohe = OneHotEncoder(drop='first')
X_train_gender_city = ohe.fit_transform(X_train[['gender','city']])
X_test_gender_city = ohe.fit_transform(X_test[['gender','city']])

X_train_gender_city.shape

(80, 4)

In [ ]:
#Extracting Age
X_train_age = X_train[['age']].values # Keep as DataFrame then extract values
X_test_age = X_test[['age']].values # Keep as DataFrame then extract values
X_train_age.shape

(80, 1)

Concanating all the things

In [ ]:
print("Shape of X_train_age:", X_train_age.shape)
print("Shape of X_train_fever:", X_train_fever.shape)
print("Shape of X_train_gender_city (dense):", X_train_gender_city.toarray().shape)
print("Shape of X_train_cough:", X_train_cough.shape)


X_train_transformed = np.concatenate((X_train_age,X_train_fever,X_train_gender_city.toarray(),X_train_cough),axis=1)
X_test_transformed = np.concatenate((X_test_age,X_test_fever,X_test_gender_city.toarray(),X_test_cough),axis=1)

X_train_transformed.shape

Shape of X_train_age: (80, 1)
Shape of X_train_fever: (80, 1)
Shape of X_train_gender_city (dense): (80, 4)
Shape of X_train_cough: (80, 1)


(80, 7)

In [ ]:
from sklearn.compose import ColumnTransformer

with column transformer

In [ ]:
transformer = ColumnTransformer(transformers=[
    ('tnf1',SimpleImputer(),['fever']), # Corrected column name
    ('tnf2',OrdinalEncoder(categories=[['Mild','Strong']]),['cough']),
    ('tnf3',OneHotEncoder(drop='first'),['gender','city'])

],remainder='passthrough')

In [ ]:
transformer.fit_transform(X_train).shape

(80, 7)

In [ ]:
transformer.fit_transform(X_test).shape

(20, 7)